In [1]:
import numpy as np
import pandas as pd
from pulp import *

# 读取作物信息表，仅包含作物编号和作物名称两列
crop_info = pd.read_excel('作物信息（附件1表2）.xlsx', usecols=['作物编号', '作物名称'])

# 去除空格
crop_info.columns = [col.strip() for col in crop_info.columns]
crop_info['作物名称'] = crop_info['作物名称'].str.strip()

# 删除包含“羊肚菌”下方的无效数据
crop_info = crop_info.dropna(subset=['作物编号', '作物名称'])  # 删除空值行
crop_info = crop_info[crop_info['作物编号'].apply(lambda x: str(x).isdigit())]  # 仅保留作物编号为数字的行

# 建立作物名称到作物编号的映射关系
crop_mapping = dict(zip(crop_info['作物名称'], crop_info['作物编号']))

merged_data = pd.read_excel('2023产量表.xlsx')
total_yield_by_crop = merged_data.groupby('作物名称')['总产量/斤'].sum().reset_index()


# 根据每块地的面积设置权重 S ，根据每种地的类型设置特定的种植矩阵
data = pd.read_excel('乡村现有耕地（附件1表1）.xlsx')
S = data['地块面积/亩']

data = pd.read_excel('作物收入汇总.xlsx')

# print(data[['作物编号', '是否豆类']].dropna().head())  # 检查前几行数据，确保没有缺失值
O = data['亩产量/斤']
N = data['作物']
is_bean = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# 读取2023年种植情况
planting_2023 = {
    'A': {
        'A1': [6], 'A2': [7], 'A3': [7], 'A4': [1], 'A5': [4], 'A6': [8]
    },
    'B': {
        'B1': [6], 'B2': [2], 'B3': [3], 'B4': [4], 'B5': [5], 'B6': [8],
        'B7': [6], 'B8': [8], 'B9': [9], 'B10': [10], 'B11': [1], 'B12': [7],
        'B13': [14], 'B14': [15]
    },
    'C': {
        'C1': [11], 'C2': [12], 'C3': [1], 'C4': [13], 'C5': [6], 'C6': [3]
    },
    'D': {
        'D1': [20, 36], 'D2': [28, 35], 'D3': [21, 35], 'D4': [22, 35],
        'D5': [17, 36], 'D6': [18, 37], 'D7': [16], 'D8': [16]
    },
    'E': {
        'E1': [18, 38], 'E2': [24, 38], 'E3': [25, 38], 'E4': [26, 39],
        'E5': [28, 39], 'E6': [27, 39], 'E7': [19, 40], 'E8': [19, 40],
        'E9': [18, 40], 'E10': [17, 41], 'E11': [17, 41], 'E12': [22, 41],
        'E13': [21, 41], 'E14': [29, 41], 'E15': [30, 27, 41], 'E16': [31, 41]
    },
    'F': {
        'F1': [32, 33, 24, 21], 'F2': [25, 26, 22, 29], 'F3': [17, 28, 30],
        'F4': [19, 34, 23]
    }
}

# print(planting_2023)

# N:作物名称
N_A = N[:15].values
N_B = N[15:30].values
N_C = N[30:45].values

N_D_1 = N.iloc[45:64].values
N_D_2 = N.iloc[82:85].values
# 拼接
N_D = np.concatenate([N_D_1, N_D_2])

N_E_1 = N.iloc[64:82].values
N_E_2 = N.iloc[85:89].values
N_E = np.concatenate([N_E_1, N_E_2])

N_F = N[89:125].values

# S:地块面积

S_A = S[0:6].values
# print("A:",S_A)
S_B = S[6:20].values
# print("B:",S_B)
S_C = S[20:26].values
# print("C:",S_C)
S_D = S[26:34].values
# print("D:",S_D)
S_E = S[34:50].values
# print("E:",S_E)
S_F = S[50:54].values
# print("F:",S_F)
# print(len(S_A),len(S_B),len(S_C),len(S_D),len(S_E),len(S_F))

# O：地块作物产出
O_A = O[:15].values
O_B = O[15:30].values
O_C = O[30:45].values

O_D_1 = O.iloc[45:64].values
O_D_2 = O.iloc[82:85].values
# 拼接
O_D = np.concatenate([O_D_1, O_D_2])

O_E_1 = O.iloc[64:82].values
O_E_2 = O.iloc[85:89].values
O_E = np.concatenate([O_E_1, O_E_2])

O_F = O[89:125].values


# r_A:地块A能种的作物的利润
r_A = data['每亩利润'][:15].values # 平旱地
r_B = data['每亩利润'][15:30].values # 梯田
r_C = data['每亩利润'][30:45].values # 山坡地

r_D_1 = data['每亩利润'].iloc[45:64].values  # 第一个区段（45到63，包含 64 不包括）
r_D_2 = data['每亩利润'].iloc[82:85].values  # 第二个区段（83到84，包含 85 不包括）
r_D = np.concatenate([r_D_1, r_D_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二

r_E_1 = data['每亩利润'].iloc[64:82].values
r_E_2 = data['每亩利润'].iloc[85:89].values
r_E =  np.concatenate([r_E_1, r_E_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二 # 普通大棚 = 普通大棚一 + 普通大棚二

r_F = data['每亩利润'][89:125].values # 智慧大棚 = 智慧大棚一 + 智慧大棚二

# print(r_A,r_B,r_C,r_D,r_E,r_F)


problem = LpProblem("Maximize_Profit", LpMaximize)

# 定义变量：作物种植比例w_X（连续变量）和作物种植决策z_X_i（0-1变量）
# w_A[i][j] 表示地块 A 的第 i 块地对于第 j 种作物的种植比例，而 z_A[i][j] 则是其是否种植的决策变量
# w'_A 是 0 到 10 之间的整数,从而控制w_X范围[0, 1],且为0.1倍数
# 0 到 10 之间的整数变量（用于定义 w_X_prime）

# 定义2024-2030年每年的变量
years = range(2024, 2031)

def create_variables(prefix, S, N, years):
    w_prime = {year: [[LpVariable(f"{prefix}_prime_{year}_{i}_{j}", 0, 10, cat='Integer') for j in range(len(N))] for i in range(len(S))] for year in years}
    w = {year: [[LpVariable(f"{prefix}_{year}_{i}_{j}", 0, 1) for j in range(len(N))] for i in range(len(S))] for year in years}
    z = {year: [[LpVariable(f"z_{prefix}_{year}_{i}_{j}", 0, 1, cat="Binary") for j in range(len(N))] for i in range(len(S))] for year in years}
    return w_prime, w, z


w_A_prime, w_A, z_A = create_variables("A", S_A, N_A, years)
w_B_prime, w_B, z_B = create_variables("B", S_B, N_B, years)
w_C_prime, w_C, z_C = create_variables("C", S_C, N_C, years)
w_D_prime, w_D, z_D = create_variables("D", S_D, N_D, years)
w_E_prime, w_E, z_E = create_variables("E", S_E, N_E, years)
w_F_prime, w_F, z_F = create_variables("F", S_F, N_F, years)


# 添加 w_X_prime 与 w_X 之间的关系约束
def add_w_constraints(problem, w_prime, w, S, N, years):
    for year in years:
        for i in range(len(S)):
            for j in range(len(N)):
                problem += w[year][i][j] == w_prime[year][i][j] * 0.1

add_w_constraints(problem, w_A_prime, w_A, S_A, N_A, years)
add_w_constraints(problem, w_B_prime, w_B, S_B, N_B, years)
add_w_constraints(problem, w_C_prime, w_C, S_C, N_C, years)
add_w_constraints(problem, w_D_prime, w_D, S_D, N_D, years)
add_w_constraints(problem, w_E_prime, w_E, S_E, N_E, years)
add_w_constraints(problem, w_F_prime, w_F, S_F, N_F, years)


# 定义目标函数：最大化2024-2030年的总利润
problem += lpSum([
    lpSum([
        S_A[i] * lpSum([r_A[j] * w_A[year][i][j] for j in range(len(r_A))]) for i in range(len(S_A))
    ] + [
        S_B[i] * lpSum([r_B[j] * w_B[year][i][j] for j in range(len(r_B))]) for i in range(len(S_B))
    ] + [
        S_C[i] * lpSum([r_C[j] * w_C[year][i][j] for j in range(len(r_C))]) for i in range(len(S_C))
    ] + [
        S_D[i] * lpSum([r_D[j] * w_D[year][i][j] for j in range(len(r_D))]) for i in range(len(S_D))
    ] + [
        S_E[i] * lpSum([r_E[j] * w_E[year][i][j] for j in range(len(r_E))]) for i in range(len(S_E))
    ] + [
        S_F[i] * lpSum([r_F[j] * w_F[year][i][j] for j in range(len(r_F))]) for i in range(len(S_F))
    ]) for year in years
])


# 添加约束：每块地的种植比例之和必须小于等于1
def add_area_constraints(problem, w, S, years):
    for year in years:
        for i in range(len(S)):
            problem += lpSum(w[year][i]) <= 1

add_area_constraints(problem, w_A, S_A, years)
add_area_constraints(problem, w_B, S_B, years)
add_area_constraints(problem, w_C, S_C, years)
add_area_constraints(problem, w_D, S_D, years)
add_area_constraints(problem, w_E, S_E, years)
add_area_constraints(problem, w_F, S_F, years)


# 添加 w_i <= z_i 约束
def add_w_z_constraints(problem, w, z, S, N, years):
    for year in years:
        for i in range(len(S)):
            for j in range(len(N)):
                problem += w[year][i][j] <= z[year][i][j]

add_w_z_constraints(problem, w_A, z_A, S_A, N_A, years)
add_w_z_constraints(problem, w_B, z_B, S_B, N_B, years)
add_w_z_constraints(problem, w_C, z_C, S_C, N_C, years)
add_w_z_constraints(problem, w_D, z_D, S_D, N_D, years)
add_w_z_constraints(problem, w_E, z_E, S_E, N_E, years)
add_w_z_constraints(problem, w_F, z_F, S_F, N_F, years)


crop_name_list = total_yield_by_crop['作物名称']
yield_limit_list = total_yield_by_crop['总产量/斤']
# 为每种作物添加产量约束，确保其在所有地块上的总产量不超过预期产量
# 使用 numpy 的 where 来查找 crop_name 在 N_X 中的位置
def get_crop_index(crop_name, N_X):
    result = np.where(N_X == crop_name)[0]
    if len(result) > 0:
        return result[0]  # 返回找到的第一个位置
    else:
        return -1  # 如果未找到，返回 -1



# 使用 numpy 的 where 来查找 crop_name 在 N_X 中的位置
def get_crop_index(crop_name, N_X):
    result = np.where(N_X == crop_name)[0]
    return result[0] if len(result) > 0 else -1

# 产量约束
for year in years:
    for crop_idx in range(len(total_yield_by_crop)):
        crop_name = total_yield_by_crop['作物名称'][crop_idx]
        yield_limit = total_yield_by_crop['总产量/斤'][crop_idx]
        total_yield_crop = lpSum([
            lpSum([S_A[i] * w_A[year][i][get_crop_index(crop_name, N_A)] * O_A[get_crop_index(crop_name, N_A)] if crop_name in N_A else 0 for i in range(len(S_A))]) +
            lpSum([S_B[i] * w_B[year][i][get_crop_index(crop_name, N_B)] * O_B[get_crop_index(crop_name, N_B)] if crop_name in N_B else 0 for i in range(len(S_B))]) +
            lpSum([S_C[i] * w_C[year][i][get_crop_index(crop_name, N_C)] * O_C[get_crop_index(crop_name, N_C)] if crop_name in N_C else 0 for i in range(len(S_C))]) +
            lpSum([S_D[i] * w_D[year][i][get_crop_index(crop_name, N_D)] * O_D[get_crop_index(crop_name, N_D)] if crop_name in N_D else 0 for i in range(len(S_D))]) +
            lpSum([S_E[i] * w_E[year][i][get_crop_index(crop_name, N_E)] * O_E[get_crop_index(crop_name, N_E)] if crop_name in N_E else 0 for i in range(len(S_E))]) +
            lpSum([S_F[i] * w_F[year][i][get_crop_index(crop_name, N_F)] * O_F[get_crop_index(crop_name, N_F)] if crop_name in N_F else 0 for i in range(len(S_F))])
        ])
        problem += (total_yield_crop <= yield_limit, f"Total_yield_constraint_for_{crop_name}_{year}")

# D地块的约束条件
for year in years:
    for i in range(len(S_D)):
        # 约束1: 如果种植水稻(索引0),则只能种植水稻;否则可以种植其他作物
        problem += w_D[year][i][0] == 0 or lpSum(w_D[year][i]) == 1

        # 定义第一季蔬菜(索引1-18)和第二季蔬菜(最后3个索引)的种植比例总和
        first_season_veg = lpSum(w_D[year][i][1:19])
        second_season_veg = lpSum(w_D[year][i][-3:])

        # 约束2: 水稻与第一季蔬菜互斥
        problem += w_D[year][i][0] + first_season_veg <= 1

        # 约束3: 水稻与第二季蔬菜互斥
        problem += w_D[year][i][0] + second_season_veg <= 1

        # 约束4: 每块地每年最多种植3种作物，先不限制
        problem += lpSum(z_D[year][i]) <= 3

        # 约束5: 第二季蔬菜的种植比例总和不超过1
        problem += second_season_veg <= 1

        # 约束6: 第二季蔬菜只能在种植了第一季蔬菜的地块上种植
        problem += lpSum(w_D[year][i][-3:]) <= first_season_veg

# E地块的约束条件
for year in years:
    for i in range(len(S_E)):
        # 定义第一季蔬菜(前18个)和第二季菌菇类(后面的)的种植比例总和
        first_season_veg = lpSum(w_E[year][i][:18])
        second_season_mushroom = lpSum(w_E[year][i][18:])

        # 约束1: 第一季蔬菜的种植比例总和不超过1
        problem += first_season_veg <= 1

        # 约束2: 第二季菌菇类的种植比例总和不超过1
        problem += second_season_mushroom <= 1

        # 约束3: 每块地每年最多种植3种作物
        problem += lpSum(z_E[year][i]) <= 3

        # 约束4: 第二季菌菇类只能在种植了第一季蔬菜的地块上种植
        problem += second_season_mushroom <= first_season_veg

# F地块的约束条件
for year in years:
    for i in range(len(S_F)):
        # 约束1: 每种作物在每年只能种植一季(前18个为第一季,后18个为第二季)
        for j in range(18):
            problem += z_F[year][i][j] + z_F[year][i][j + 18] <= 1

        # 约束2: 第一季最多种植3种作物
        problem += lpSum(z_F[year][i][:18]) <= 3
        # 约束3: 第二季最多种植3种作物
        problem += lpSum(z_F[year][i][18:]) <= 3

        # 约束4:  第一季的总种植比例不超过1，第二季只能种在第一季的土地上
        problem += lpSum(w_F[year][i][:18]) <= 1
        problem += lpSum(w_F[year][i][18:]) <= lpSum(w_F[year][i][:18])


# 连茬种植约束
for year in range(2024, 2031):
    # 2024年使用2023年的种植情况,其他年份使用上一年的优化结果
    # A：平旱地——单季
    if year == 2024:
        # z只能是0/1,23年已种
        problem += z_A[2024][0][5] == 0
        problem += z_A[2024][1][6] == 0
        problem += z_A[2024][2][6] == 0
        problem += z_A[2024][3][0] == 0
        problem += z_A[2024][4][3] == 0
        problem += z_A[2024][5][7] == 0
    else:
        for i in range(len(S_A)):
            for j in range(len(N_A)):
                problem += z_A[year-1][i][j] + z_A[year][i][j] <= 1

    # B：梯田——单季
    if year ==2024:
        problem += z_B[2024][0][5] == 0  #B1
        problem += z_B[2024][1][1] == 0
        problem += z_B[2024][2][2] == 0
        problem += z_B[2024][3][3] == 0
        problem += z_B[2024][4][4] == 0
        problem += z_B[2024][5][7] == 0
        problem += z_B[2024][6][5] == 0
        problem += z_B[2024][7][7] == 0
        problem += z_B[2024][8][8] == 0
        problem += z_B[2024][9][9] == 0
        problem += z_B[2024][10][0] == 0
        problem += z_B[2024][11][6] == 0
        problem += z_B[2024][12][13] == 0
        problem += z_B[2024][13][14] == 0
    else:
        for i in range(len(S_B)):
            for j in range(len(N_B)):
                problem += z_B[year - 1][i][j] + z_B[year][i][j] <= 1

    # C：山坡地——单季
    if year == 2024:
        problem += z_C[2024][0][10] == 0
        problem += z_C[2024][1][11] == 0
        problem += z_C[2024][2][0] == 0
        problem += z_C[2024][3][12] == 0
        problem += z_C[2024][4][5] == 0
        problem += z_C[2024][5][2] == 0
    else:
        for i in range(len(S_C)):
            for j in range(len(N_C)):
                problem += z_C[year - 1][i][j] + z_C[year][i][j] <= 1

    # D:水浇地——单季水稻或两季蔬菜（第一季：蔬菜除大白菜、白萝卜、红萝卜，第二季：大白菜、白萝卜、红萝卜）
    # 水浇地可种植的作物编号是16-37，因此在算是第j种作物时要-16
    if year == 2024:
        problem += z_D[2024][0][4] == 0
        problem += z_D[2024][0][20] == 0
        problem += z_D[2024][1][12] == 0
        problem += z_D[2024][1][19] == 0
        problem += z_D[2024][2][5] == 0
        problem += z_D[2024][2][19] == 0
        problem += z_D[2024][3][6] == 0
        problem += z_D[2024][3][19] == 0
        problem += z_D[2024][4][1] == 0
        problem += z_D[2024][4][20] == 0
        problem += z_D[2024][5][2] == 0
        problem += z_D[2024][5][21] == 0
        problem += z_D[2024][6][0] == 0
        problem += z_D[2024][7][0] == 0
    else:
        for i in range(len(S_D)):
            for j in range(len(N_D)):
                problem += z_D[year - 1][i][j] + z_D[year][i][j] <= 1


    # E：普通大棚——第一季蔬菜除大白菜、白萝卜、红萝卜，第二季食用菌
    # 普通大棚可种植的作物编号是17-34,38-41，因此在算是第j种作物时第一季要-17，第二季要-20
    if year == 2024:
        problem += z_E[2024][0][1] == 0
        problem += z_E[2024][0][18] == 0
        problem += z_E[2024][1][7] == 0
        problem += z_E[2024][1][18] == 0
        problem += z_E[2024][2][8] == 0
        problem += z_E[2024][2][18] == 0
        problem += z_E[2024][3][9] == 0
        problem += z_E[2024][3][19] == 0
        problem += z_E[2024][4][11] == 0
        problem += z_E[2024][4][19] == 0
        problem += z_E[2024][5][10] == 0
        problem += z_E[2024][5][19] == 0
        problem += z_E[2024][6][2] == 0
        problem += z_E[2024][6][20] == 0
        problem += z_E[2024][7][2] == 0
        problem += z_E[2024][7][20] == 0
        problem += z_E[2024][8][1] == 0
        problem += z_E[2024][8][20] == 0
        problem += z_E[2024][9][0] == 0
        problem += z_E[2024][9][21] == 0
        problem += z_E[2024][10][0] == 0
        problem += z_E[2024][10][21] == 0
        problem += z_E[2024][11][5] == 0
        problem += z_E[2024][11][21] == 0
        problem += z_E[2024][12][4] == 0
        problem += z_E[2024][12][21] == 0
        problem += z_E[2024][13][12] == 0
        problem += z_E[2024][13][21] == 0
        problem += z_E[2024][14][13] == 0 # E15第一季种了两种作物
        problem += z_E[2024][14][10] == 0
        problem += z_E[2024][14][21] == 0
        problem += z_E[2024][15][14] == 0
        problem += z_E[2024][15][21] == 0
    else:
        for i in range(len(S_E)):
            for j in range(len(N_E)):
                problem += z_E[year - 1][i][j] + z_E[year][i][j] <= 1

    # F:智慧大棚——每年可种（除大白菜、白萝卜、红萝卜）外的两季蔬菜
    # 假设一年内两季不能种相同作物，前一年第二季和今年第一季不能种相同作物，今年年第i季可以种和前一年第i季作物相同的作物
    # 作物编号是17-34，长度18，因为有两季，所以拼接成长度36，前一季第j种作物时-17，后一季第j种作物时+1
    if year == 2024:
        problem += z_F[2024][0][7] == 0 # 23年第二季种植的导致24年第一季不能种
        problem += z_F[2024][0][4] == 0
        problem += z_F[2024][1][5] == 0
        problem += z_F[2024][1][12] == 0
        problem += z_F[2024][2][11] == 0
        problem += z_F[2024][2][13] == 0
        problem += z_F[2024][3][17] == 0
        problem += z_F[2024][3][6] == 0
    else:
        for i in range(len(S_F)):
            for j in range(18):  # 前18种作物是第一季可选作物
                problem += z_F[year-1][i][j] + z_F[year-1][i][j + 18] <= 1  # 2024年同样满足本年内两季不能种相同的
                problem += z_F[year-1][i][j + 18] + z_F[year][i][j] <= 1 #25年以后每年第一季度不能和前一年第二季度种相同的



In [14]:
is_bean = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0,0, 1, 1, 1, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0]

# is_bean_X:地块X可种植的作物列表是否为豆类
is_bean_A = is_bean[:15] # is_bean[0]~is_bean[14]
is_bean_B = is_bean[:15]
is_bean_C = is_bean[:15]
is_bean_D = is_bean[15:37] # is_bean[15]~is_bean[36]

is_bean_E_1 = is_bean[16:34]
is_bean_E_2 = is_bean[37:41]
is_bean_E = np.concatenate([is_bean_E_1, is_bean_E_2])

is_bean_F = is_bean[16:34]
print(is_bean_F)
len(is_bean_F)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


18

In [33]:
print('N_A',N_A,"len(N_A)",len(N_A))
print('N_B',N_B,"len(N_B)",len(N_B))
print('N_C',N_C,"len(N_C)",len(N_C))
print('N_D',N_D,"len(N_D)",len(N_D))
print('N_E',N_E,"len(N_E)",len(N_E))
print('N_F',N_F,"len(N_F)",len(N_F))


N_A ['黄豆' '黑豆' '红豆' '绿豆' '爬豆' '小麦' '玉米' '谷子' '高粱' '黍子' '荞麦' '南瓜' '红薯' '莜麦'
 '大麦'] len(N_A) 15
N_B ['黄豆' '黑豆' '红豆' '绿豆' '爬豆' '小麦' '玉米' '谷子' '高粱' '黍子' '荞麦' '南瓜' '红薯' '莜麦'
 '大麦'] len(N_B) 15
N_C ['黄豆' '黑豆' '红豆' '绿豆' '爬豆' '小麦' '玉米' '谷子' '高粱' '黍子' '荞麦' '南瓜' '红薯' '莜麦'
 '大麦'] len(N_C) 15
N_D ['水稻' '豇豆' '刀豆' '芸豆' '土豆' '西红柿' '茄子' '菠菜' '青椒' '菜花' '包菜' '油麦菜' '小青菜' '黄瓜'
 '生菜' '辣椒' '空心菜' '黄心菜' '芹菜' '大白菜' '白萝卜' '红萝卜'] len(N_D) 22
N_E ['豇豆' '刀豆' '芸豆' '土豆' '西红柿' '茄子' '菠菜' '青椒' '菜花' '包菜' '油麦菜' '小青菜' '黄瓜' '生菜'
 '辣椒' '空心菜' '黄心菜' '芹菜' '榆黄菇' '香菇' '白灵菇' '羊肚菌'] len(N_E) 22
N_F ['豇豆' '刀豆' '芸豆' '土豆' '西红柿' '茄子' '菠菜' '青椒' '菜花' '包菜' '油麦菜' '小青菜' '黄瓜' '生菜'
 '辣椒' '空心菜' '黄心菜' '芹菜' '豇豆' '刀豆' '芸豆' '土豆' '西红柿' '茄子' '菠菜' '青椒' '菜花' '包菜'
 '油麦菜' '小青菜' '黄瓜' '生菜' '辣椒' '空心菜' '黄心菜' '芹菜'] len(N_F) 36


In [34]:
print(total_yield_by_crop)

   作物名称   总产量/斤
0    刀豆   26880
1    包菜    4050
2    南瓜   35100
3    土豆   30000
4   大白菜  150000
5    大麦   10000
6   小青菜   35480
7    小麦  170840
8   榆黄菇    9000
9    水稻   21000
10  油麦菜    4500
11   爬豆    9875
12   玉米  132750
13   生菜    2850
14  白灵菇   18000
15  白萝卜  100000
16  空心菜    3600
17  红萝卜   36000
18   红薯   36000
19   红豆   22400
20   绿豆   33040
21  羊肚菌    4200
22   芸豆    6480
23   芹菜    1800
24   茄子   45360
25   荞麦    1500
26   莜麦   14000
27   菜花    3600
28   菠菜     900
29  西红柿   36210
30   谷子   71400
31   豇豆   36480
32   辣椒    1200
33   青椒    2610
34   香菇    7200
35   高粱   30000
36  黄心菜    1800
37   黄瓜   13050
38   黄豆   57000
39   黍子   12500
40   黑豆   21850


TypeError: can't multiply sequence by non-int of type 'list'